In [162]:
import pandas as pd
import numpy as np
import sqlite3

In [163]:
df = pd.read_csv('../database/alunos-ativos-da-pos-graduacao.csv', sep=';', usecols=['nome_aluno']).sample(frac=1).reset_index(drop=True)
df = df.rename(columns={'nome_aluno' : 'nome'})

pd.options.display.max_rows = 6

In [164]:
#Atribuindo sexo considerando nomes com maior frequencia
df_nomes_frequencia = df['nome'].map(lambda y: y.split()[0]).value_counts()

nomes_masculinos = ['Lucas', 'João', 'Gabriel', 'Matheus', 'Rafael', 'Guilherme', 'Gustavo', 'Bruno', 'Luiz', 'Paulo', 'Pedro', 'André', 'José', 'Rodrigo', 'Thiago', 'Felipe', 'Daniel', 'Marcelo', 'Eduardo']
nomes_femininos = ['Ana', 'Maria', 'Mariana', 'Bruna', 'Gabriela', 'Camila', 'Fernanda', 'Amanda', 'Aline', 'Leticia', 'Larissa', 'Juliana', 'Jéssica', 'Beatriz', 'Marina']

df['sexo'] = None

df.loc[df['nome'].str.split().str[0].isin(nomes_masculinos), 'sexo'] = 'M'
df.loc[df['nome'].str.split().str[0].isin(nomes_femininos), 'sexo'] = 'F'

lista_sexo_aleatorio = np.random.choice(['M', 'F'], size=df['sexo'].isnull().sum())
df.loc[df['sexo'].isnull(), 'sexo'] = lista_sexo_aleatorio

In [165]:
#Define data de nascimento e idade
df['nascimento'] = None
df.loc[df['nascimento'].isnull(), 'nascimento'] = np.random.choice(pd.date_range(start='1910-01-01', end='2019-12-31').strftime('%Y-%m-%d'), size=len(df))

df['idade'] = (pd.Timestamp.now() - pd.to_datetime(df['nascimento'])).dt.days // 365

In [166]:
#define altura com influencia da idade e sexo
df['altura'] = np.random.randint(120, 210, size=len(df))
df.loc[df['idade'] <= 18, 'altura'] -= (25 * (1 - (df['idade'] / 18) ** 2 )).round() #quanto menor a idade, maior o decrescimo
df.loc[df['idade'] >= 60, 'altura'] -= (30 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round() #quanto maior a idade, maior o decrescimo

df.loc[df['sexo'] == 'F', 'altura'] -= np.random.randint(1, 8)
df.loc[df['sexo'] == 'M', 'altura'] += np.random.randint(1, 8)

In [167]:
#define o peso com influencia da idade e sexo
df['peso'] = np.round(np.random.uniform(25.0, 150, size=len(df)), decimals=2)
df.loc[df['idade'] <= 18, 'peso'] -= (8 * (1 - (df['idade'] / 18) ** 2 )).round(decimals=2)
df.loc[df['idade'] >= 60, 'peso'] -= (15 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round(decimals=2)

df.loc[df['sexo'] == 'F', 'peso'] -= (df['peso'] * 0.05).round(decimals=2)
df.loc[df['sexo'] == 'M', 'peso'] += (df['peso'] * 0.05).round(decimals=2)

In [168]:
# define pressao arterial
df['pressao-arterial-sistolica'] = 10 * np.random.randint(5, 23, size=len(df))
df['pressao-arterial-diastolica'] = 10 * np.random.randint(3, (df['pressao-arterial-sistolica']/10), size=len(df))

df.loc[df['idade'] <= 18, ['pressao-arterial-sistolica', 'pressao-arterial-diastolica']] -= (10 * np.random.randint(1, 4))

In [169]:
# define glicemia com aumento na terceira idade
df['glicemia'] = 5 * np.random.randint(8, 44, size=len(df))
df.loc[df['idade'] >= 60, 'glicemia'] += 5 * np.random.randint(1, 4)

In [170]:
# Gera um valor de frequencia cardiaca com influencia da idade
df['frequencia-cardiaca'] = np.random.randint(30, 210, size=len(df))

df.loc[df['idade'] <= 18, 'frequencia-cardiaca'] += (np.random.randint(20, 40) * (1 - (df['idade'] / 18) ** 2 )).round()
df.loc[df['idade'] >= 60, 'frequencia-cardiaca'] += (np.random.randint(10, 30) * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round()

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica,glicemia,frequencia-cardiaca
0,Matheus Ramos de Carvalho,M,1994-05-20,30,138,108.77,70,40,210,106
1,Letícia Gabriele Ribeiro Rocumba,M,1987-04-12,37,166,119.48,220,160,160,63
2,Vinícius José de Oliveira Garcia,M,1976-01-20,48,207,44.18,70,50,140,107
...,...,...,...,...,...,...,...,...,...,...
3882,Felipe Adriano Alves de Oliveira,M,2004-08-16,19,138,152.25,120,80,65,67
3883,Ana Eliza de Jesus,F,1966-02-27,58,169,81.50,120,50,190,32
3884,Argos Yoneda Coletti,F,1965-10-20,58,116,140.97,200,140,150,131


In [171]:
conexao = sqlite3.connect('../pagina/db.sqlite3')

df.to_sql('Pacientes', conexao, if_exists='replace', index=False)

3885

In [172]:
"""
df['circunferencia-cintura']
df['diabetes']
df['doenca-cronica']
df['tabagismo']
df['colesterol']
df['funcao-renal']
df['alergia']
df['duracao-sono']"""

"\ndf['circunferencia-cintura']\ndf['diabetes']\ndf['doenca-cronica']\ndf['tabagismo']\ndf['colesterol']\ndf['funcao-renal']\ndf['alergia']\ndf['duracao-sono']"